In [1]:
import tensorflow as tf
import mlflow
from PIL import Image
import os
import base64
import io
import numpy as np
from transformers import ViTFeatureExtractor

from dotenv import load_dotenv
load_dotenv()

2025-06-22 06:45:48.152631: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-22 06:45:48.185027: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750549548.208795    6771 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750549548.217182    6771 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-22 06:45:48.253776: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

True

# Load Utils

In [2]:
mlflow.set_tracking_uri(os.getenv("MLFOW_TRACKING_SERVER_URL"))
mlflow.set_experiment(os.getenv("EXPERIMENT_NAME"))

<Experiment: artifact_location='wasbs://model-artifacts@rsscmodelartifacts.blob.core.windows.net/1', creation_time=1750529138044, experiment_id='1', last_update_time=1750529138044, lifecycle_stage='active', name='RSSC', tags={}>

In [3]:
best_resnet50v2_uri =  f"runs:/{os.getenv("MLFLOW_BEST_RESNET50V2_RUNID")}/model"
best_convnexttiny_uri = f"runs:/{os.getenv("MLFLOW_BEST_CONVNEXTTINY_RUNID")}/model"
best_vit_uri = f"runs:/{os.getenv("MLFLOW_BEST_VIT_RUNID")}/model"

models = {
    "Best ResNet50v2": mlflow.tensorflow.load_model(best_resnet50v2_uri), 
    "Best ConvNeXt-Tiny": mlflow.tensorflow.load_model(best_convnexttiny_uri), 
    "Best ViT Patch16 In1k": mlflow.tensorflow.load_model(best_vit_uri) 
}

fe = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")

KeyboardInterrupt: 

Unable to stream download: HTTPSConnectionPool(host='rsscmodelartifacts.blob.core.windows.net', port=443): Read timed out.


In [ ]:
def predict_img_vit(model, image_data_b64):
    id2label = {
        0: 'airplane', 1: 'cloud', 10: 'ship', 11: 'airport', 12: 'river', 13: 'golf_course', 14: 'roundabout', 15: 'church', 16: 'circular_farmland', 17: 'overpass', 18: 'railway', 19: 'wetland', 2: 'mountain', 20: 'lake', 21: 'parking_lot', 22: 'intersection', 23: 'tennis_court', 24: 'runway', 25: 'industrial_area', 26: 'chaparral', 27: 'bridge', 28: 'sparse_residential', 29: 'freeway', 3: 'medium_residential', 30: 'sea_ice', 31: 'beach', 32: 'palace', 33: 'snowberg', 34: 'meadow', 35: 'ground_track_field', 36: 'harbor', 37: 'rectangular_farmland', 38: 'island', 39: 'basketball_court', 4: 'thermal_power_station', 40: 'desert', 41: 'stadium', 42: 'forest',43: 'storage_tank', 44: 'railway_station', 5: 'terrace', 6: 'commercial_area', 7: 'dense_residential', 8: 'baseball_diamond', 9: 'mobile_home_park'
    }
    
    decoded = base64.b64decode(image_data_b64)
    image = Image.open(io.BytesIO(decoded)).convert("RGB")    
    inputs = fe(images=image, return_tensors="tf")
    outputs = model({"pixel_values": inputs["pixel_values"]}, training=False)
    logits = outputs["logits"] if isinstance(outputs, dict) else outputs
    

    probs = tf.nn.softmax(logits, axis=-1).numpy()[0]
    predicted_class_idx = int(tf.argmax(probs))
    acc = float(probs[predicted_class_idx])

    predicted_label = id2label[predicted_class_idx]
    
    return (
        predicted_label, 
        acc * 100
    )

def predict_img_cnn(model, image_data_b64):
    CLASSES = ['airplane', 'airport', 'baseball_diamond', 'basketball_court', 'beach', 'bridge', 'chaparral', 'church', 'circular_farmland', 'cloud', 'commercial_area', 'dense_residential', 'desert', 'forest', 'freeway', 'golf_course', 'ground_track_field', 'harbor', 'industrial_area', 'intersection', 'island', 'lake', 'meadow', 'medium_residential', 'mobile_home_park', 'mountain', 'overpass', 'palace', 'parking_lot', 'railway', 'railway_station', 'rectangular_farmland', 'river', 'roundabout', 'runway', 'sea_ice', 'ship', 'snowberg', 'sparse_residential', 'stadium', 'storage_tank', 'tennis_court', 'terrace', 'thermal_power_station', 'wetland']
    
    decoded = base64.b64decode(image_data_b64)
    image = Image.open(io.BytesIO(decoded)).convert("RGB")
    image = image.resize((224, 224))  
    img_array = tf.keras.preprocessing.image.img_to_array(image)
    img_array = tf.expand_dims(img_array, 0)  # add batch dim
    img_array = img_array / 255.0
    
    logits = model.predict(img_array)[0]
    probs = tf.nn.softmax(logits, axis=-1).numpy()
    idx = np.argmax(probs)
    
    return (
        CLASSES[idx], 
        float(probs[idx]) * 100, 
    )

In [ ]:
img_p = "../../datasets/CLEAN_NWPU-RESISC45-SAMPLE500-SPLITTED8515/test/desert/desert_023.jpg"

with open(img_p, "rb") as img_f:
    img_b64 = base64.b64encode(img_f.read())

resnet_label, resnet_acc = predict_img_cnn(models["Best ResNet50v2"], img_b64)
convnext_label, convnext_acc = predict_img_cnn(models["Best ConvNeXt-Tiny"], img_b64)
vit_label, vit_acc = predict_img_vit(models["Best ViT Patch16 In1k"], img_b64)

print(f"ResNet50v2: {resnet_label} ({resnet_acc:.2f}%)")
print(f"ConvNeXt-Tiny: {convnext_label} ({convnext_acc:.2f}%)")
print(f"ViT: {vit_label} ({vit_acc:.2f}%)")